In [17]:
import os
import time
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def scarpe(auction_id):
    path = f"https://www.centralcarauctions.com/auction/{auction_id}"
    
    options = ChromeOptions()
    options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    
    # Accept cookies
    try:
        cookie_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "ccc-notify-accept"))
        )
        cookie_btn.click()
        print("🍪 Cookie accepted")
        time.sleep(1)
    except:
        print("No cookie popup")
    
    # Login (skip if not needed)
    try:
        driver.find_element(By.XPATH, './/a[text()="Login"]').click()
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,'username'))).send_keys("fourbrotherstrading@icloud.com")
        driver.find_element(By.ID, 'password').send_keys("Sultanmirza1501#")
        driver.find_element(By.XPATH, './/button[text()="Sign in"]').click()
        time.sleep(2)
        driver.back()
        driver.back()
    except:
        print("Login skipped or failed")
    
    # Total cars
    try:
        cars_text = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, './/div[@class="pagination-pages"]/span[last()]'))
        ).text.strip("()")
        total_cars = int(cars_text.split(" ")[0])
        print(f"Total cars: {total_cars}")
    except:
        print("Cannot find total cars")
        driver.quit()
        return
    
    # HTML folder
    html_folder = os.path.join(os.getcwd(), "html")
    os.makedirs(html_folder, exist_ok=True)
    
    car_count = 0

    while car_count < total_cars:
        try:
            cars_list = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.card.card-vehicle'))
            )
            
            for car in cars_list:
                if car_count >= total_cars:
                    break

                # Open car in new tab
                car_link = car.get_attribute("href")
                driver.execute_script("window.open(arguments[0]);", car_link)
                driver.switch_to.window(driver.window_handles[1])

                # Wait only for main container
                try:
                    WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.main-container"))
                    )
                except:
                    pass  # still grab HTML even if slow
                
                # Save HTML
                filename = f"car_{car_count + 1}.html"
                try:
                    full_html = driver.page_source
                    with open(os.path.join(html_folder, filename), "w", encoding="utf-8") as f:
                        f.write(full_html)
                    print(f"✅ Saved full HTML: {filename}")
                except Exception as e:
                    print(f"❌ Could not save HTML {filename}: {e}")
                
                car_count += 1

                # Close tab and return
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            # Load more
            try:
                next_btn = driver.find_element(By.XPATH, './/a[text()="Load More"]')
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(2)
                print("Load More clicked")
            except:
                break

        except Exception as e:
            print("Main loop error:", e)
            break

    driver.quit()
    print(f"Scraped {car_count} cars")

# Example usage
scarpe(839)


🍪 Cookie accepted
Total cars: 89
✅ Saved full HTML: car_1.html
✅ Saved full HTML: car_2.html
✅ Saved full HTML: car_3.html
✅ Saved full HTML: car_4.html
✅ Saved full HTML: car_5.html
✅ Saved full HTML: car_6.html
✅ Saved full HTML: car_7.html
✅ Saved full HTML: car_8.html
✅ Saved full HTML: car_9.html
✅ Saved full HTML: car_10.html
✅ Saved full HTML: car_11.html
✅ Saved full HTML: car_12.html
✅ Saved full HTML: car_13.html
✅ Saved full HTML: car_14.html
✅ Saved full HTML: car_15.html
✅ Saved full HTML: car_16.html
✅ Saved full HTML: car_17.html
✅ Saved full HTML: car_18.html
✅ Saved full HTML: car_19.html
✅ Saved full HTML: car_20.html
✅ Saved full HTML: car_21.html
✅ Saved full HTML: car_22.html
✅ Saved full HTML: car_23.html
✅ Saved full HTML: car_24.html
✅ Saved full HTML: car_25.html
✅ Saved full HTML: car_26.html
✅ Saved full HTML: car_27.html
✅ Saved full HTML: car_28.html
✅ Saved full HTML: car_29.html
✅ Saved full HTML: car_30.html
Load More clicked
✅ Saved full HTML: car_31.ht

In [2]:
import os
import csv
from bs4 import BeautifulSoup

html_folder = os.path.join(os.getcwd(), "html")
html_files = [f for f in os.listdir(html_folder) if f.endswith(".html")]
previous_data ={"Auction Name": ""}
csv_file = os.path.join(os.getcwd(), "data.csv")

# Define headers
headers = ["Auction Name",
           "Title",
           "Make", 
           "Model", 
           "Variant",
           "Center",
           "Start Date",
           "Start Time",
           "Lot",
           "Cap Clean",
           "Cap Avg",
           "Cap Retail",
           "Autotrader Retail Value",
           "Inspection Report",
           "Tyres Condition",
           "General Condition",
           "Grade",
           "Mileage Warranted",
           "Images",
           "Non Runner",
           "Additional information",
           "Damaged_Images",
           "Damage_Details",
           ]


all_data = []

for html_file in html_files:
    path = os.path.join(html_folder, html_file)
    with open(path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        
        data = {} 
        
        make_tag = soup.find("h1")
        make = make_tag.get_text(strip=True) if make_tag else ""
        data["Make"] = make
        
        p_tag = soup.find("p")
        full_text = p_tag.get_text(strip=True) if p_tag else ""

        model = ""
        variant = ""

        if full_text:
            parts = full_text.split()
            
            if parts[0][0].isdigit():
                if len(parts) > 1 and parts[1][0].isdigit() and len(parts) > 2:
                    model = " ".join(parts[:3])
                    variant = " ".join(parts[3:]) if len(parts) > 3 else ""
                else:
                    model = " ".join(parts[:2])
                    variant = " ".join(parts[2:]) if len(parts) > 2 else ""
            else:
                model = parts[0]
                variant = " ".join(parts[1:]) if len(parts) > 1 else ""

        
        data["Model"] = model
        data["Variant"] = variant
        
        # Full title
        data["Title"] = f"{make} {full_text}".strip()
        
        # Centers
        centersElement = soup.find("p", class_="auction-location")
        centersText=centersElement.get_text(strip=True) if centersElement else ""
        data["Center"] = centersText
        
        previous_auction_name = previous_data.get("Auction Name", "")

        divCarDetails = soup.find("div", class_="auction-details")
        if divCarDetails:
            h2_tag = divCarDetails.find("h2")
            if h2_tag:
                data["Auction Name"] = h2_tag.get_text(strip=True)
            else:
                data["Auction Name"] = previous_auction_name or ""
        else:
            data["Auction Name"] = previous_auction_name or ""

        # Update previous_data for next file
        previous_data["Auction Name"] = data["Auction Name"]
        
        
        
        auction_sale_div = soup.find("div", class_="auction-sale")
        sale_date = ""
        sale_time = ""
        lot_number = ""
        row = ""

        if auction_sale_div:
            p_tags = auction_sale_div.find_all("p")
            for p in p_tags:
                text = p.get_text(strip=True)
                if text.startswith("Sale date:"):
                    sale_date = text.replace("Sale date:", "").strip()
                elif text.startswith("Sale time:"):
                    sale_time = text.replace("Sale time:", "").strip()
                elif text.startswith("Lot number:"):
                    lot_number = text.replace("Lot number:", "").strip()
                elif text.startswith("Row:"):
                    row = text.replace("Row:", "").strip()

        data["Start Date"] = sale_date
        data["Start Time"] = sale_time
        data["Lot"] = lot_number
 

        price_list_cap = soup.find("div", class_="price-list")
        price_p_tag = price_list_cap.find_all("p") if price_list_cap else None
        cap_clean = price_p_tag[0].get_text(strip=True).replace("Cap Clean:", "") if price_p_tag else ""
        cap_avg = price_p_tag[1].get_text(strip=True).replace("Cap Avg.:", "") if price_p_tag else ""
        cap_retail = price_p_tag[2].get_text(strip=True).replace("Cap Retail:", "") if price_p_tag else ""
        price_footer = soup.find("div", class_="price-footer")
        autotrader_retail = ""

        if price_footer:
            p_tag = price_footer.find("p")
            if p_tag:
                texts = [t.strip() for t in p_tag.stripped_strings if t.strip() != "Autotrader Retail Value:"]
                if texts:
                    autotrader_retail = texts[-1] 
        
        data["Cap Clean"] = cap_clean
        data["Cap Avg"] = cap_avg
        data["Cap Retail"] = cap_retail
        data["Autotrader Retail Value"] = autotrader_retail
        
        download_a_tag = soup.find("a", string=lambda x: x and "Download inspection report" in x)
        inspection_report_link = ""
        
        if download_a_tag and download_a_tag.has_attr("href"):
            inspection_report_link = download_a_tag["href"]
        base_url = "https://www.centralcarauctions.com"  
        full_link_inspection_report = f"{base_url}{inspection_report_link}" if inspection_report_link.startswith("/") else inspection_report_link

        data["Inspection Report"] = full_link_inspection_report
        
        summary_div = soup.find("div", class_="auction-summary")

        table_data = {} 

        summary_div = soup.find("div", class_="auction-summary")
        table_data = {}

        if summary_div:
            tables = summary_div.find_all("table")
            for table in tables:
                for row in table.find_all("tr"):
                    tds = row.find_all("td")
                    if len(tds) >= 2:
                        key = tds[0].get_text(strip=True).replace(":", "")
                        strong_tag = tds[1].find("strong")
                        value = strong_tag.get_text(separator=" ", strip=True) if strong_tag else tds[1].get_text(strip=True)
                        if key.lower() == "mileage":
                            import re
                            match = re.search(r'\d+', value.replace(",", ""))
                            value = int(match.group()) if match else value
                        elif key.lower() in ["engine size", "cc"]:
                            match = re.search(r'(\d+)', value)
                            if match:
                                value = round(int(match.group()) / 1000, 1) 
                            else:
                                value = value
                        
                        table_data[key] = value
                        
                        if key not in headers:
                            headers.append(key)

        data.update(table_data)

        tyres_condition = []


        tyres_div = soup.find("div", class_="col-md-6", text=lambda x: x and "Tyres" in x)

        if not tyres_div:  
            for d in soup.find_all("div", class_="col-md-6"):
                h4 = d.find("h4")
                if h4 and "Tyres" in h4.get_text(strip=True):
                    tyres_div = d
                    break


        if tyres_div:
            table = tyres_div.find("table")
            if table:
                rows = table.find_all("tr")[1:] 
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 3:
                        tyre_name = cols[0].get_text(strip=True)
                        make = cols[1].get_text(strip=True) or ""  
                        condition = cols[2].get_text(strip=True) or ""  
                        tyres_condition.append(f"{tyre_name}|{make}|{condition}")

        data["Tyres Condition"] = ", ".join(tyres_condition)
        
        damage_list = []
        cond_div = soup.find("div", id="inspection")
        if cond_div:
            for cap in cond_div.find_all("div", style=lambda x: x and "position:absolute" in x):
                lines = [x.strip() for x in cap.get_text("\n", strip=True).split("\n") if x.strip()]
                if len(lines) >= 2:
                    part = lines[0]  
                    issue1 = lines[1]  
                    issue2 = lines[2] if len(lines) > 2 else "" 
                    damage_list.append(f"{part}|{issue1}|{issue2}")

        data["General Condition"] = ", ".join(damage_list)
        
        images_list = []

        gallery = soup.find("div", class_="auction-gallery")

        if gallery:
            imgs = gallery.find_all("img")
            for img in imgs:
                src = img.get("src")
                if src:
                    if "keys" in src:
                        parts = src.split("/motorvehicle/")[1].split("/")
                        vehicle_id = parts[0]  # 74879
                        file_name = parts[-1].split("---")[0].split(".")[0]  # 1763130111_gzHFaR

                        src = f"https://cca-web-public.s3.eu-west-1.amazonaws.com/motorvehicle/{vehicle_id}/{file_name}.jpg"
                    images_list.append(src)

        data["Images"] = ", ".join(images_list)


        
        damage_images = []
        damage_details = []

        # Sirf wo gallery jisme images + damage details hotay hain
        gallery = soup.find("div", class_="gallerywidget-condition")
        if gallery:
            slides = gallery.find_all("div", class_="slick-slide")

            for slide in slides:
                img = slide.find("img")
                detail_box = slide.find("div")  # text box yahi hota hai

                # ⭐ IMAGE URL CLEAN KARO
                if img:
                    src = img.get("src")
                    if src:
                        if "key=" in src:
                            src = src.split("key=")[-1].split("&")[0]
                    damage_images.append(src)

                # ⭐ DAMAGE DETAILS
                if detail_box:
                    strong = detail_box.find("strong")
                    part = strong.get_text(strip=True) if strong else ""

                    lines = detail_box.get_text(separator="\n", strip=True).split("\n")
                    if lines and lines[0] == part:
                        lines = lines[1:]

                    cond = lines[0] if len(lines) > 0 else ""
                    size = lines[1] if len(lines) > 1 else ""

                    damage_details.append(f"{part}|{cond}|{size}")


        data["Damaged_Images"] = ", ".join(damage_images)
        data["Damage_Details"] = ", ".join(damage_details)
        
        
        img = soup.find('img', class_='auction-grade')

        grade = None
        if img and img.has_attr('src'):
            src = img['src']  
            import re
            m = re.search(r'(\d+)(?=\.[a-zA-Z]+$)', src)  
            if m:
                grade = m.group()


        data["Grade"] = grade if grade else ""
        
        
        mileage_td = soup.find('td', string=lambda x: x and "Mileage" in x)
        warranted = "No"

        if mileage_td:
            value_td = mileage_td.find_next('td')
            if value_td and "Warranted" in value_td.get_text(strip=True):
                warranted = "Yes"
        
        data["Mileage Warranted"] = warranted
        
        cat = ""
        cat_text = ""
        Additional_information = {}
        badge = soup.find("span", string=lambda x: x and "Cat:" in x)
        if badge:
            cat = badge.get_text(strip=True)  

        summary_div = soup.find("div", id="summary")
        if summary_div:
            p = summary_div.find("p")
            if p:
                cat_text = p.get_text(strip=True)

        Additional_information["Category"] = {
            "Cat": cat,
            "Info": cat_text
        }
        data["Additional information"] = Additional_information
        
        
        non_runner_status = "No"
        badge = soup.find("span", class_="badge badge-danger badge-lg")
        if badge:
            text = badge.get_text(strip=True).lower()
            if "non runner" in text:
                non_runner_status = "Yes"
        data["Non Runner"] = non_runner_status
                
        all_data.append(data)


# Write CSV
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for row in all_data:
        writer.writerow(row)

print(f"\nAll data saved to {csv_file}")


C:\Users\HP\AppData\Local\Temp\ipykernel_16108\3243549716.py:190: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tyres_div = soup.find("div", class_="col-md-6", text=lambda x: x and "Tyres" in x)



All data saved to d:\shakeeb\CCA update\data.csv


In [3]:
import pandas as pd
df = pd.read_csv("data.csv")
df = df.rename(columns={
    "Fuel": "Fuel Type",
    "Body": "Body Type",
    "Engine size": "CC",
    "Owners": "Former Keepers",
    "Number of Services": "No of Services",
    "MOT": "MOT Expiry Date",
    "Registration": "Reg",
    "Reg year": "D.O.R",
    "Cap Avg": "CAP Average",
    "Cap Retail": "CAP Retail",
    "Cap Clean": "CAP Clean",
})

df.to_csv("cca_data.csv", index=False)


In [1]:
import threading,requests,os
import pandas as pd
from urllib.parse import urlparse, urljoin
from PIL import Image, ImageDraw, ImageFont


def add_watermark_to_image(image_path, text="Sourced from Central Car Auction"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # 🔹 Slightly smaller dynamic font size (half previous)
        font_size = max(20, image.width // 50)  # reduced from //25 → //50
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = int(font_size * 0.6)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_padding = int(font_size * 0.4)
        draw.rectangle(
            [x - box_padding, y - box_padding, x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 180)
        )

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 240))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")

    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")



def download_images(data, main_folder="Images", column_name="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for _, row in data.iterrows():
        reg_no = str(row["Reg"]).strip().replace("/", "_").replace("\\", "_")
        image_urls = row.get(column_name)

        if not image_urls or pd.isna(image_urls):
            print(f"⚠️ Skipping {reg_no}: no {column_name.lower()}.")
            continue

        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)

        urls = [u.strip() for u in image_urls.split(",") if u.strip()]
        for idx, url in enumerate(urls, start=1):
            if "centralcarauctions.com/motorvehicledamageimage" in url:
                url = url.split("key=")[-1].split("&")[0]

            if not url.startswith(("http://", "https://")):
                url = "https://" + url.lstrip("/")

            parsed = urlparse(url)
            if not parsed.scheme or not parsed.netloc:
                print(f"⚠️ Invalid URL skipped: {url}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=15)
                response.raise_for_status()

                ext = os.path.splitext(parsed.path)[1] or ".jpg"
                save_path = os.path.join(reg_folder, f"{idx}{ext}")

                with open(save_path, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                add_watermark_to_image(save_path)
                print(f"✅ Saved: {save_path}")

            except Exception as e:
                print(f"❌ Failed to download {url} for {reg_no}: {e}")



def download_reports(data, main_folder="Inspection Report"):
    os.makedirs(main_folder, exist_ok=True)

    for _, row in data.iterrows():
        reg_no = str(row["Reg"]).strip().replace("/", "_").replace("\\", "_")
        report_url = row.get("Inspection Report")

        if not report_url or pd.isna(report_url):
            print(f"⚠️ Missing Inspection Report for {reg_no}")
            continue

        if not report_url.startswith(("http://", "https://")):
            report_url = "https://" + report_url.lstrip("/")

        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        save_path = os.path.join(reg_folder, f"{reg_no}.pdf")

        try:
            response = requests.get(report_url, stream=True, timeout=15)
            response.raise_for_status()
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"✅ Downloaded report: {save_path}")
        except Exception as e:
            print(f"❌ Failed to download {report_url} for {reg_no}: {e}")



def start_funcs():
    df = pd.read_csv("cca_data.csv")
    reports = df[["Reg", "Inspection Report"]]
    reg_img = df[["Reg", "Images"]]
    cond_img = df[["Reg", "Damaged_Images"]]

    threads = [
        threading.Thread(target=download_reports, args=(reports,)),
        threading.Thread(target=download_images, args=(reg_img, "Images", "Images")),
        threading.Thread(target=download_images, args=(cond_img, "Damaged_Images", "Damaged_Images")),
    ]

    for t in threads:
        t.start()
    for t in threads:
        t.join()


if __name__ == "__main__":
    start_funcs()


✅ Downloaded report: Inspection Report\NM53EET\NM53EET.pdf
✅ Downloaded report: Inspection Report\LC11CZB\LC11CZB.pdf
✅ Downloaded report: Inspection Report\ML09DKN\ML09DKN.pdf
✅ Downloaded report: Inspection Report\SG15BBZ\SG15BBZ.pdf
✅ Watermark added to Damaged_Images\NM53EET\1.jpg
✅ Saved: Damaged_Images\NM53EET\1.jpg
✅ Downloaded report: Inspection Report\DE14RVO\DE14RVO.pdf
✅ Downloaded report: Inspection Report\OY63WSV\OY63WSV.pdf
✅ Downloaded report: Inspection Report\MJ60ENE\MJ60ENE.pdf
✅ Downloaded report: Inspection Report\HJ10KJF\HJ10KJF.pdf
✅ Watermark added to Damaged_Images\NM53EET\2.jpg
✅ Saved: Damaged_Images\NM53EET\2.jpg
⚠️ Skipping LC11CZB: no damaged_images.
⚠️ Skipping ML09DKN: no damaged_images.
⚠️ Skipping SG15BBZ: no damaged_images.
✅ Downloaded report: Inspection Report\MA66NUC\MA66NUC.pdf
✅ Downloaded report: Inspection Report\SC18NZN\SC18NZN.pdf
✅ Watermark added to Images\NM53EET\1.jpg
✅ Saved: Images\NM53EET\1.jpg
✅ Downloaded report: Inspection Report\CP1